<a href="https://colab.research.google.com/github/usm-cos-432/InClass/blob/master/chapter6/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR 10 Lab

We explore convolutional model development for the CIFAR 10 data set.
https://www.cs.toronto.edu/~kriz/cifar.html

Note : When you open a new Colab from Github (like this one), you cannot save changes. So it's usually best to store the Colab in you personal drive "File > Save a copy in drive..." before you do anything else.

In [ ]:
!pip install d2l==0.14.4


In [ ]:
from d2l import torch as d2l
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler
from matplotlib import pyplot as plt

In [ ]:
def load_data_cifar10(batch_size, resize=None): 
    """Download the cifar10 dataset and then load it into memory."""
    transformer = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    if resize:
        transformer.insert(0, transforms.Resize(resize))
    
    cifar10_train = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transformer)
    cifar10_test = torchvision.datasets.CIFAR10(root="../data", train=False, transform=transformer, download=True)
    indices = list(range(len(cifar10_train)))
    np.random.shuffle(indices)
    split = int(np.floor(0.20 * len(cifar10_train)))

    cifar10_train_sample = SubsetRandomSampler(indices[split:])
    cifar10_valid_sample = SubsetRandomSampler(indices[:split])


    return (data.DataLoader(cifar10_train, batch_size, sampler=cifar10_train_sample),
            data.DataLoader(cifar10_train, batch_size, sampler=cifar10_valid_sample),
            data.DataLoader(cifar10_test, batch_size, shuffle=False))

In [ ]:
def get_cifar10_labels(labels):  
    """Return text labels for the CIFAR10 dataset."""
    text_labels = ['plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    return [text_labels[int(i)] for i in labels]

In [ ]:
# modified show_images to unnormalize tensors and move RGB channels to beginning
def show_images(imgs, num_rows, num_cols, titles=None, scale=2.5):  #@save
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        npimg = np.transpose(npimg,(1,2,0)) 
        ax.imshow(npimg)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [ ]:
batch_size=24

train_iter, valid_iter, test_iter = load_data_cifar10(batch_size)
X, y = next(iter(train_iter))

show_images(X, 2, 9, titles=get_cifar10_labels(y));


In [ ]:
print(X.shape)
print(len(train_iter.sampler))
print(len(valid_iter.sampler))

## Model

As compared with our concise implementation
of softmax regression implementation
(:numref:`sec_softmax_concise`),
the only difference is that we add
*two* fully-connected layers
(previously, we added *one*).
The first is our hidden layer,
which contains 256 hidden units
and applies the ReLU activation function.
The second is our output layer.


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()



The training loop is exactly the same
as when we implemented softmax regression.
This modularity enables us to separate
matters concerning the model architecture
from orthogonal considerations. 

train_ch6 code was changed to include train loss & train accuracy in plot


In [ ]:
def train_ch6(net, train_iter, test_iter, num_epochs, lr,
              device=d2l.try_gpu()):
    """Train a model with a GPU (defined in Chapter 6)."""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            torch.nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[0, num_epochs],
                            # ylim=[0.0, 3.0],
                            legend=['train loss', 'train acc', 'test acc'])
    timer = d2l.Timer()
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = d2l.Accumulator(3)
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            net.train()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_loss = metric[0]/metric[2]
            train_acc = metric[1]/metric[2]
            if (i + 1) % 50 == 0:
                animator.add(epoch + i / len(train_iter),
                             (train_loss, train_acc, None))
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter, device)
        animator.add(epoch+1, (train_loss, train_acc, test_acc))
    print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

In [ ]:
batch_size, lr, num_epochs = 256, 0.1, 5

In [ ]:
train_iter, valid_iter, test_iter = load_data_cifar10(batch_size)

train_ch6(net, train_iter, valid_iter, num_epochs, lr)

In [ ]:
print(d2l.evaluate_accuracy_gpu(net, train_iter))
print(d2l.evaluate_accuracy_gpu(net, test_iter))
